<img style="float: right;"  src="images/LogoP.jpg" width="200">

# Linear AC 03 - Impulse Response

This document deals with the **Impulse Response**, this the response of a circuit to an special signal that can be used to obtain the response to any other signal.

Version 1.0 (7/4/2019)  
License information is at the end of the document

---
**Bill Of Materials (BOM):**

* One $100nF$ Capacitor
* One $47k \Omega$ Resistor

---

## Capacitors and linearity

In the previous tutorial we have described the **capacitor** and used its equation to solve the output of a circuit against two different inputs. Now, we want to ease obtaining the solution of any circuit tgat contains capacitors for any possible input signal. In order to do that, we will apply some mathematical linear tricks.

We have described the capacitor in the previous chapter. The question that arises is. Is the capacitor linear?. And the answer is a **qualified** yes. If you remember, the capacitor current $i_C$ depends on the capacitor voltage as shown in the equation:

$$i_C = C \frac{d v_C}{d t}$$

This equation is linear. Multiplying by a constant $C$ is a linear operation and diferentiating is also a linear operation. Let's see now the equation that obtains the $v_C$ voltage:

$$v_C = V_C(0) + \frac{1}{C} \int^t_0 i_C \cdot dt$$

This equation **is not linear**. Multiplying by $1/C$ is linear, integrating is linear, but addind the $V_C(0)$ constant **is not linear**. This is problem, because we need linear components to have linear circuits. And we need linear circuit to apply to them all the mathematical tricks that only work with linear functions.

There are two solutions to this problem.

The **first solution** is wisely choosing the zero time. The numbers on the time line are completely arbitrary. We can put the zero time whenever we choose. Just set the zero time when you know that the the capacitor voltage $v_C$ is zero. Easy peasy. Problem solved.

This first solution don't always work. Sometimes the voltage is **never** zero. Sometimes we have several capacitors and they are never zero at the same time.

The **second solution** always work and consists in detaching the capacitor from its initial conditions. Any capacitor with non zero initial conditions $V_C(0)$ is equivalent to a zero initial conditions capacitor in series with an independent source that carries the capacitor's initial conditions.

![Equivalent](images\Linear_AC_02\initial_conditions.png)

So, if needed, we can have all capacitors with zero initial conditions if we use an additional independent voltage source for each capacitor whose initial conditions are not zero.


## LTI Systems

Circuits that you can build using independent sources and linear components, whether they have **state** or not, can be included in the **Linear Time-Independent (LTI)** system cathegory.

An **LTI** system is **linear** so its outputs (voltages and currents) are linear functions of its inputs (independent sources) but also is **Time-Independent**. That means that its behavior is insensitive to a time translation. 

As an example, if we have a **LTI** system $f$ and feed it with an imput $x(t)$ that changes with time we will get an output $y(t)$ that changes with time:

$$x(t) \overset{f}{\longrightarrow} y(t)$$

This can also be expressed as:

$$y(t) = f(x(t))$$

Then, if we shift the input in time by a $t_0$ amount, the output will shift by the same amount:

$$x(t-t_0) \overset{f}{\longrightarrow} y(t-t_0) \qquad y(t-t_0)=f(x(t-t_0))$$

Having an **LTI** system permits describing the system using its **Impulse Response**. To ease the explanations we will consider in this document systems that only have one input and one output. As **initial conditions** are equivalent to having additional inputs, we will also consider zero initial conditions.

The explanations that follows could be quite hard to grasp so we will  use practical examples.

---

![Practical Icon](images/pt.png)

---

## SLab Setup

First we will need to import the **SLab** module

In [ ]:
# Import the main SLab module
import slab

Then we need to connect with the board.

In [ ]:
boardFolder = ''                                # Board folder (leave '' if you use only one board)
slab.setFilePrefix('../Files/')                 # Set File Prefix
slab.setCalPrefix('Calibrations/'+boardFolder)  # Set Calibration Prefix         
slab.connect()                                  # Connect to the board

It is always a good idea to check the board operation.
The following cell checks the calibration for the DAC channels and the first four ADC channels.

In [ ]:
# Check the calibration
slab.checkCalibration(pause=False,na=4,nm=10)

## Example LTI system

The **LTI** system we will use is the same **RC** circuit we use in the previous tutorial.

![RC](images\Linear_AC_02\circuit2.png)

We only have one input $V_{DAC1}$ and we will consider only one output $V_{ADC2}$ that corresponds with the capacitor voltage $v_C$. Also we will always start our measurements with zero initial conditions.

First, we will repeat the response to one step in the input. In this case, for simplicity,  we will use a step change from $0V$ to $1V$ in the circuit input.  This input is the **Unity Step Function** also known as the **Heaviside Step Function**. As this function is used a lot, so it has its own short name $u(t)$.

As we have analyzed the circuit, we know what we will get:

$$v_C = 1V \left( 1 -  e^{-\frac{t}{\tau}} \right)$$

This is the **step response** $s(t)$ for the **step input** $u(t)$

We can obtain the **step response** by measuring it too and see that it agrees with our formula.


In [ ]:
# Import the arange and exp functions from numpy
from numpy import arange,exp

# Calculate the step response

tau = 0.0047                    # Time constant is 4.7ms
t  = arange(0.0,0.05,0.0001)    # Time from 0 to 50ms in 0.1ms increments
vc = 1.0*(1-exp(-t/tau))        # Calculate capacitor voltage

# Measure the step response

slab.setSampleTime(0.0001)           # Set sample time to 0.1ms
slab.setTransientStorage(625,2)      # Set storage for 625 samples of ADC1 and ADC2

# Perform the step plot with a change of DAC1 from 0V to 1V 
# By default, DAC1 will be at the start voltage for 1s before the change
data = slab.stepResponse(0.0,1.0)     
tm  = data[0]  # Get time from data
vcm = data[2]  # Get ADC2 from data

# Plot vc against time in both cases
slab.plotnn([t,tm],[vc,vcm],'RC Step Response'
            ,'time [s]','vc [V]',['Calculated','Measured'])

The **step response** $s(t)$ is very important because it completelly describes the circuit operation. We can use it to obtain the response of the circuit against any other input. So, we will store this important information in a  $s$ vector.

In [ ]:
# Store the step response in a vector
s = vc

## Using step response for fun an profit

So, we know that $s(t)$ is the circuit's response to $u(t)$ let's express that this way:

$$u(t) \overset{f}{\longrightarrow} s(t)$$

We can also define our circuit **LTI System** as a function $f$ that transforms the input $u(t)$ in the output $s(t)$.

$$s(t) = f(u(t))$$

We know tha the circuit $f$ function transfoms the input $u(t)$ in the output $s(t)$ but we don't know yet how it can trsnsform any other input.

Now, we want to use our step response $s(t)$ to obtain the response to a signal similar to the one that ended last chapter. To make things a little simpler, we will use a signal that starts at zero value.

In [ ]:
from numpy import linspace

# Define a special waveform to be applied on DAC1
v = []
for sample in range(0,250):
    if sample < 100:
        v.append(0.0)
    else:    
        if sample > 160:
            v.append(0.5)
        else:
            v.append(2.0)
   
# Define a time vector to show the vaveform
time = linspace(0,0.025,250)            

# Plot the waveform
slab.plot11(time,v,'Waveform','time [s]','Voltage [V]')

This signal that we will call $x_1(t)$, although much more complex than the step input $u(t)$, can be generated by **superposition** and **time shifting** of the step $u(t)$ signal. We can do superposition because we have a **Linear System** and we can do **time shifting** because we have a **Time-Invariant** system.

$$x_1(t) = 2V \cdot u(t - 10ms) - 1.5V \cdot u(t-16ms)$$

So, as $u(t)$ has a $s(t)$ response, $x_1(t)$ shall have a response $y_1(t)$:

$$u(t) \overset{f}{\longrightarrow} s(t)$$

$$x_1(t) \overset{f}{\longrightarrow} y_1(t)$$

So, we can calculate $y_1(t)$:

$$y_1(t) = 2V \cdot s(t-10ms) - 1.5V \cdot s(t-16ms)$$

We want to try that, so we will define a **shift** function to shift a vector.

In [ ]:
import numpy as np

# Define shift function
def shift(vector,offset):
    out = []
    for i in range(0,len(vector)):
        if i < offset:
            out.append(0.0)
        else:
            out.append(vector[i-offset])
    return np.array(out) 

And we will use this function to calculate the circuit $y_1(t)$ response from the step $s(t)$ response.

In [ ]:
# Obtain y1(t) from s(t)
y1 = 2.0*shift(s,100)-1.5*shift(s,160)

# Plot y1(t)
slab.plot11(t,y1,'','time [s]','y1 [V]')

We can compare that against a measurement

In [ ]:
slab.loadWavetable(v)                # Upload the waveform of 250 points
slab.setSampleTime(0.0001)           # Set sample time to 0.1ms (full wave will be 25ms)
slab.setTransientStorage(250,2)      # Set storage for 250 samples of ADC1 and ADC2

data1 = slab.waveResponse()          # Obtain circuit's response to v

slab.plotnn([t,data1[0]],[y1,data1[2]],''
            ,'time [s]','Response [V]',['Calculated','Measured'])

You can think that you can use $u(t)$ to syntetize $x_1(t)$ because this signal was defined by constant intervals, but we can syntetize **any function** just using the $u(t)$ function. Let's try with that $x_2(t)$ function:

In [ ]:
# Define a new waveform to be applied on DAC1
x2 = []
for sample in range(0,500):
    if sample < 100:
        x2.append(0.0)
    else:    
        x2.append(2*np.sin((sample-100)/40)*exp(-(sample-100)/100)+(1-exp(-(sample-100)/100)))
        
# Define a time vector to show the vaveform
time = linspace(0,0.05,500)            

# Plot the waveform
slab.plot11(time,x2,'Waveform','time [s]','Voltage [V]')        

We can feed that function to our system and obtain its $y_2(t)$ response:

In [ ]:
slab.loadWavetable(x2)               # Upload the waveform of 250 points
slab.setSampleTime(0.0001)           # Set sample time to 0.1ms (full wave will be 25ms)
slab.setTransientStorage(500,2)      # Set storage for 500 samples of ADC1 and ADC2

data2 = slab.waveResponse()          # Obtain circuit's response to v
tm =  data2[0]                       # Get time
vim = data2[1]                       # Get vi(t) from ADC1
vcm = data2[2]                       # Get vc(t) from ADC2

slab.plot1n(tm,[vim,vcm],'Circuit response'
            ,'time [s]','Voltage [V]',['Input','Output'])

In order to obtain $y_2(t)$ from $s(t)$ we need to generate $x_2(t)$ from $u(t)$. As our syntesis won't be perfect we will call it $x'_2(t)$

$$ x'_2(t) \approx x_2(t)$$

To generate $x'_2(t)$ we define time intervals $\Delta t$ of, for instance, $2ms$. And, for each time interval we define a small piece of the function.

$$x'_2(t) = \sum_{i=0}^{n-1} x_2(i\cdot \Delta t) 
\cdot (u(t-i\cdot \Delta t)-u(t-(i+1)\cdot \Delta t))$$

In [ ]:
x2a = []                       # Define an empty list
for i in range(0,25):          # Use 25 sections
    for z in range(0,20):      
        x2a.append(x2[i*20])   # Define wave for each section 
x2a = np.array(x2a)            # Convert list to numpy array

# Plot the original and the approximation
slab.plot1n(time,[x2,x2a],'Waveform approximation'
            ,'time [s]','Voltage [V]',['x2(t)','x\'2(t)'])   

We see that $x'_2(t)$ is a non perfect approximation of $x_2(t)$.

As $x'_2(t)$ has been generated using $u(t)$ we can use the step response $s(t)$ to obtain the system's $y'_2(t)$ response

$$u(t) \overset{f}{\longrightarrow} s(t)$$

$$x'_2(t) \overset{f}{\longrightarrow} y'_2(t)$$

Where $y'_2(t)$ can be obtained:

$$y'_2(t) = \sum_{i=0}^{n-1} x_2(i\cdot \Delta t) 
\cdot (s(t-i\cdot \Delta t)-s(t-(i+1)\cdot \Delta t))$$

We will use the following code cell to test that.

In [ ]:
y2a = np.zeros(500)            # Define start vector filled with zeros
for i in range(0,25):          # Use 25 sections
    # Compute output from the s(t) in this section
    y2a = y2a + x2[i*20]*shift(s,20*i) - x2[i*20]*shift(s,20*(i+1))
y2a = np.array(y2a)            # Convert list to numpy array

# Plot the obtained response
slab.plot11(time,y2a,'Calculated response','time [s]','y2a [V]')

And we can compare it with the measured $y_2(t)$

In [ ]:
# Plot measured y2(t) and calculated y2a(t)
slab.plot1n(time,[y2a,vcm],'Calculation vs Measurement'
            ,'time [s]','Voltage [V]',['Calculated','Measured'])

It's good, but ot is not perfect.

## Going for perfection

In order to improve our approximation to the real $y_2(t)$ we can use a smaller $\Delta t$ time. As our measurement system is a **discrete sampled** system the smaller $\Delta t$ we can use is the **sample time**, $0.1ms$ in our case. Let's use that to obtain the response $y''_2(t)$ of our system:

In [ ]:
# Compute response by using s(t) on each sample
y2aa = np.zeros(500)
for i in range(0,500):          # Use 500 sections
    y2aa = y2aa + x2[i]*shift(s,i) - x2[i]*shift(s,i+1)
y2aa = np.array(y2aa)            # Convert list to numpy array

# Plot measured y2(t) and calculated y2aa(t)
slab.plot1n(time,[y2aa,vcm],'Calculation vs Measurement'
            ,'time [s]','Voltage [V]',['Calculated','Measured'])

As you can see the approximation is as good as you can get.

## Impulse response

In the previous sections we have obtained the circuit's response $y(t)$ by using a summation of **step responses** $s(t)$.

$$x'(t) = \sum_{i=0}^\infty x(i\cdot \Delta t) 
\cdot (s(t-i\cdot \Delta t)-s(t-(i+1)\cdot \Delta t))$$

The summation starts at $i=0$ because our $x_2(t)$ was only defined for positive times.  In general for any $x(t)$ function the summarion shall include negative values.

$$y(t) = \sum_{i=-\infty}^\infty x(i\cdot \Delta t) 
\cdot (s(t-i\cdot \Delta t)-s(t-(i+1)\cdot \Delta t))$$

---

The $s(t)$ sustraction is interesting so we will have a closer look.

$$s(t-i\cdot \Delta t)-s(t-(i+1) \cdot \Delta t)$$

As the $t-i \cdot \Delta t$ is repeated, let's call it $t_A$

$$t_A = t-i \cdot \Delta t$$

So we get:

$$s(t_A)-s(t_A - \Delta t)$$

This is what the $s$ function has increased in a $\Delta t$ time from time $t_A$, so it can be approximated by:

$$s(t_A)-s(t_A - \Delta t) \approx \frac{d}{dt} s(t_A) \cdot \Delta t$$

Now we can recover what $t_A$ contains:

$$\frac{d}{dt} s(t-i \cdot \Delta t) \cdot \Delta t$$

This approximation will very good for very small $\Delta t$ values.

The derivative of the **step response** $s(t)$ is the **impulsional response** $h(t)$. Take it, fow now, as a definition.

$$h(t) = \frac{d}{dt}s(t)$$

---

Let's return to our sum:

$$y(t) = \sum_{i=-\infty}^\infty x(i\cdot \Delta t) 
\cdot (s(t-i\cdot \Delta t)-s(t-(i+1)\cdot \Delta t))$$

It can be rewritten as:

$$y(t) = \sum_{i=-\infty}^\infty x(i\cdot \Delta t) \frac{d}{dt} s(t-i \cdot \Delta t) \cdot \Delta t $$
 
And, using the **impulsional response** $h(t)$ we get:

$$y(t) = \sum_{i=-\infty}^\infty x(i\cdot \Delta t) \; h(t - i\cdot \Delta t) \; \Delta t $$

To have a **perfect** approximation we can have $\Delta t$ infinitelly small so we get $d\tau$ instead of $\Delta t$ and we get an integral instead of a sum.

$$y(t) = \int^\infty _{-\infty}  x(\tau) \; h(t-\tau) \; d \tau$$

Observe that we integrate in $\tau$ not $t$ because the result is a function of $t$. Remember that in our previous sum we had $t$ and $i \cdot \Delta t$. We have just replaced $\Delta t$ by $d \tau$ and $i \cdot \Delta t$ by $\tau$.

The above integral is a very important one. It defines the **Convolution Operator** ($*$).

$$y(t) = \int^\infty _{-\infty}  x(\tau) \; h(t-\tau) \; d \tau
= (x * s)(t) $$

## Testing the impulse response

We can now test the impulse response $h(t)$ on our circuit. As we know:

$$h(t) = \frac{d \: s(t)}{dt}$$

We know the step response $s(t)$ of our circuit:

$$s(t) = 1V \left( 1 -  e^{-\frac{t}{\tau}} \right)$$

So, the impulse response will be:

$$h(t) = \frac{d}{dt} \left( 1 -  e^{-\frac{t}{\tau}} \right) 
= \frac{1}{\tau}e^{-\frac{t}{\tau}}$$

We can use the following code cell to calculate the impulse response for our circuit.  
The code cell also computes $h(t)$ from the measured $s(t)$. As we are using discrete measurements, we use the **difference** instead of the derivative.

$$h(i \cdot \Delta t) = \frac{1}{\Delta t} \bigl( s(i\cdot \Delta t) -s((i-1) \cdot \Delta t) \bigr)$$

At the end, the code compares the $h(t)$ obtained from the circuit equatuions with the one obtained from the measurement.

In [ ]:
# Calculate the impulse response

t  = arange(0.0,0.05,0.0001)    # Time from 0 to 50ms in 0.1ms increments
h = (1/tau)*exp(-t/tau)         # Calculate capacitor voltage

hm = (s[1:] - s[:-1])/0.0001    # Difference

# Plot calculation against measurement
# As the difference has one item less than the measurement, we need to use different time vectors
slab.plotnn([t,t[:-1]],[h,hm],'Impulse Response'
           ,'time [s]','h(t) [v]',['Calculated','Measurement'])

Now, we can apply the **convolution** to obtain the response of the circuit:

$$y(t) = (x*h)(t)$$

In our calculations, as they are discrete data, we are using the **discrete convolution** instead of the **integral convolution**. The definition of the discrete convolution is:

$$(x * h)_i = \sum_{j=0}^{n-1} x_j \; h(i - j)$$

If we compare that with the previous calculations we see that we miss a $\Delta t$ factor. So we will compute the output as:

$$y_i = (x * h)_i \cdot \Delta t$$

The following code cell obtains the response of the circuit from the **convolution** of the **input** and the **impulsional response** and compares it with the measurements.

In [ ]:
# Perform the convolution
# We need to multiply bu the sample time to get proper results
y2c = np.convolve(h,x2)*0.0001

# We get only the first 500 points as the input signal is not
# defined beyond this point
y2c = y2c[0:500]

# Now we define a time vector with the same size
t = arange(0.0,0.05,0.0001)

# Plot measured y2(t) and calculated y2aa(t)
slab.plot1n(time,[y2c,vcm],'Calculation vs Measurement'
            ,'time [s]','Voltage [V]',['Calculated','Measured'])

## Changing the order

Is interesting to note that the convolution operator is **Commutative**, so, we can change the operands order:

$$(x * s)(t) = (s * x)(t)$$

As we see **LTI Systems** can be defined by the **Convolution** of the input and the **Impulsional Response** of the system. And that convolution is **Commutative**.

If we have one system $f$ that transforms an input $x(t)$ in an output $y_1(t)$ and we feed that output in another system $g$, it ill generate an output $y_2(t)$ that depends on the **cascade** if tge systems $f$ and $g$.

$$x(t) \overset{f}{\longrightarrow} y_1(t) \overset{g}{\longrightarrow} y_2(t)$$

We can obtain $y_1(t)$ and $y_2(t)$:

$$y_1(t) = f(x(t)) \qquad y_2(t) = g(y_1(t)) = g(f(x(t)))$$

Using the impulsional responses $h_f(t)$ and $h_g(t)$ of the systems $f$ and $g$ we get:

$$y_1(t) = f(x(t)) = x(t)*h_f(t)
 \qquad y_2(t) = g(y_1(t)) = g(f(x(t))) = (x * h_f * h_g)(t)$$

But, as convolution is conmmutative, it is also true:

$$y_2(t) = (x * h_g * h_f)(t) = f(g(x(t)))$$

So, this corresponds to the system:

$$x(t) \overset{g}{\longrightarrow} y_1(t) \overset{f}{\longrightarrow} y_2(t)$$

So, if we have two linear systems in **cascade** we can change their order and the final result won't change. The intermediate results will be different, however.

$$x(t) \overset{f}{\longrightarrow} y_1(t) \overset{g}{\longrightarrow} y_2(t)$$

$$x(t) \overset{g}{\longrightarrow} y_3(t) \overset{f}{\longrightarrow} y_2(t)$$

![Cascade Systems](images\Linear_AC_02\systems.png)

Also note that the cascade combination of the systems $f$ and $g$ can be represented by a single system whose impulsional response $h_{fg}(t)$ is:

$$h_{fg}(t) = (h_f * h_g)(t)$$

## Going Infinite

Previously we have defined the **impulsional response** as the derivative of the **step response**

$$h(t) = \frac{d}{dt} s(t)$$

The question is, if the **impulsional response** is a response. To what input does it respond? Let's advance that it responds to the [Dirac Delta](https://en.wikipedia.org/wiki/Dirac_delta_function) $\delta(t)$ function. 

$$h(t) = f(\delta(t))$$

$$\delta(t) \overset{f}{\longrightarrow} h(t)$$

The question is, how is this function?

As we remember, the **step response** $s(t)$ is the response of the system $f$ to the **unity step function u(t)**:

$$s(t) = f(u(t))$$

So, the origina impulsional response:

$$h(t) = \frac{d}{dt} s(t)$$

Can be written as:

$$h(t) = \frac{d}{dt} f(u(t))$$

As the detivative is **linear** and **time-invariant** we get:

$$u(t) \overset{f}{\longrightarrow} \overset{\frac{d}{dt}}{\longrightarrow} h(t)$$

We know that **LTI Systems** are definited by a **convolution** and that this operation is **commutative**, so:

$$u(t)  \overset{\frac{d}{dt}}{\longrightarrow} \overset{f}{\longrightarrow} h(t)$$

$$h(t) = f \left( \frac{d}{dt} u(t) \right)$$

So, as we defined $\delta (t)$ as the input to the sysyem $f$ that gives $h(t)$ as response,:

$$h(t) = f(\delta(t))$$

By simple identification we get:

$$\delta(t) = \frac{d}{dt} u(t)$$

Here things get hairy becase the step function is **discontinuous** and not **derivable** at time zero. Leaving aside the mathematical formalities we can say that [Dirac Delta](https://en.wikipedia.org/wiki/Dirac_delta_function) $\delta(t)$ is a function that is zero for any time that is not zero. And, at time zero, it goes to infinity so that the integral of the function goes from $0$ to $1$ when time goes from $0^-$ to $0^+$.

![Dirac Delta](images\Linear_AC_02\delta.png)

## The important bits

To recap and not lose ourselves in this forest, we have defined the **impulsional response** $h(t)$ of a **LTI system** $f$ as the response of this system to the **Dirac Delta** $\delta (t)$ function.

$$\delta(t) \overset{f}{\longrightarrow} h(t)$$

Using this **impulsional response** we can obtain the response $y(t)$ of the system $f$ to any arbitrary input $x(t)$ using the convolution operator:

$$y(t) =  (x * s)(t) = \int^\infty _\infty  x(\tau) \; h(t-\tau) \; d \tau$$

But we cannot directly obtain the **impulse response** from measurements because the **Dirac Delta ** function $\delta(t)$ goes to infity and is impossible to implement. So we get the response $s(t)$ to the **unity step function** that is easier to implement and we get the **impulsional** response by calculating the derivative of the **step response**.

$$u(t) \overset{f}{\longrightarrow} s(t)$$

$$h(t) = \frac{d}{dt} s(t)$$

There is just one **"small"** problem. Any **stareless LTI system** wil respond instantaneously to a step change on its input and will have an unbound impulsional response. That is, is impulsional response $h(t)$ will be proportional to the **Dirac Delta** $\delta(t)$ function.

That's why we **never** use the impulsional response to describe **stateless** systems.


## Last Words

This document shows how any **Linear Time- Inveriant (LTI)** system can be described by its **impulsional response** $h(t)$. This response can be obtained from the derivative of the **unity step response**. In the case of the simple **RC** circuit it was easy to obtain the **impulsional response** from the component values because the system can be described by a simple first order differential equation.

For more complex circuits it is not that easy. You could be against a high order differential equation in no time. Also, solving the convolution integral to obtain the circuit's response to a given input is not that easy.

All that is much easy in the **S World** an alternative to the time domain where obtaining the impulsional response and solving the circuit's response to a given input is much easier.

## Document license

Copyright  ©  Vicente Jiménez (2019)  
This work is licensed under a Creative Common Attribution-ShareAlike 4.0 International license.  
This license is available at http://creativecommons.org/licenses/by-sa/4.0/

<img  src="images/cc_sa.png" width="200">